In [1]:
!pip install tensorflow tqdm tiktoken

     ---------------------------------------- 0.0/48.6 kB ? eta -:--:--
     -------------------------------- ----- 41.0/48.6 kB 960.0 kB/s eta 0:00:01
     -------------------------------------- 48.6/48.6 kB 815.7 kB/s eta 0:00:00
   ---------------------------------------- 0.0/390.3 MB ? eta -:--:--
   ---------------------------------------- 0.1/390.3 MB 2.3 MB/s eta 0:02:49
   ---------------------------------------- 0.1/390.3 MB 1.4 MB/s eta 0:04:31
   ---------------------------------------- 0.3/390.3 MB 2.0 MB/s eta 0:03:19
   ---------------------------------------- 0.5/390.3 MB 3.0 MB/s eta 0:02:11
   ---------------------------------------- 1.0/390.3 MB 4.4 MB/s eta 0:01:29
   ---------------------------------------- 1.5/390.3 MB 5.5 MB/s eta 0:01:11
   ---------------------------------------- 2.0/390.3 MB 6.3 MB/s eta 0:01:02
   ---------------------------------------- 2.5/390.3 MB 6.9 MB/s eta 0:00:57
   ---------------------------------------- 3.1/390.3 MB 7.7 MB/s eta 0:0

In [2]:
from importlib.metadata import version
print("TensorFlow version:", version("tensorflow"))
print("tqdm version:", version("tqdm"))

TensorFlow version: 2.18.0
tqdm version: 4.66.4


In [3]:
import os
import json
import urllib.request

import numpy as np
import tensorflow as tf
from tqdm import tqdm

def download_file(url, destination, backup_url=None):
    def _attempt_download(download_url):
        with urllib.request.urlopen(download_url) as response:
            # Get the total file size from headers, defaulting to 0 if not present
            file_size = int(response.headers.get("Content-Length", 0))

            # Check if file exists and has the same size
            if os.path.exists(destination):
                file_size_local = os.path.getsize(destination)
                if file_size == file_size_local:
                    print(f"File already exists and is up-to-date: {destination}")
                    return True  # Indicate success without re-downloading

            block_size = 1024  # 1 Kilobyte

            # Initialize the progress bar with total file size
            progress_bar_description = os.path.basename(download_url)
            with tqdm(total=file_size, unit="iB", unit_scale=True, desc=progress_bar_description) as progress_bar:
                with open(destination, "wb") as file:
                    while True:
                        chunk = response.read(block_size)
                        if not chunk:
                            break
                        file.write(chunk)
                        progress_bar.update(len(chunk))
            return True

    try:
        if _attempt_download(url):
            return
    except (urllib.error.HTTPError, urllib.error.URLError):
        if backup_url is not None:
            print(f"Primary URL ({url}) failed. Attempting backup URL: {backup_url}")
            try:
                if _attempt_download(backup_url):
                    return
            except urllib.error.HTTPError:
                pass

        # If we reach here, both attempts have failed
        error_message = (
            f"Failed to download from both primary URL ({url})"
            f"{' and backup URL (' + backup_url + ')' if backup_url else ''}."
            "\nCheck your internet connection or the file availability.\n"
            "For help, visit: https://github.com/rasbt/LLMs-from-scratch/discussions/273"
        )
        print(error_message)
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

def load_gpt2_params_from_tf_ckpt(ckpt_path, settings):
    # Initialize parameters dictionary with empty blocks for each layer
    params = {"blocks": [{} for _ in range(settings["n_layer"])]}

    # Iterate over each variable in the checkpoint
    for name, _ in tf.train.list_variables(ckpt_path):
        # Load the variable and remove singleton dimensions
        variable_array = np.squeeze(tf.train.load_variable(ckpt_path, name))

        # Process the variable name to extract relevant parts
        variable_name_parts = name.split("/")[1:]  # Skip the 'model/' prefix

        # Identify the target dictionary for the variable
        target_dict = params
        if variable_name_parts[0].startswith("h"):
            layer_number = int(variable_name_parts[0][1:])
            target_dict = params["blocks"][layer_number]

        # Recursively access or create nested dictionaries
        for key in variable_name_parts[1:-1]:
            target_dict = target_dict.setdefault(key, {})

        # Assign the variable array to the last key
        last_key = variable_name_parts[-1]
        target_dict[last_key] = variable_array

    return params

def download_and_load_gpt2(model_size, models_dir):
    # Validate model size
    allowed_sizes = ("124M", "355M", "774M", "1558M")
    if model_size not in allowed_sizes:
        raise ValueError(f"Model size not in {allowed_sizes}")

    # Define paths
    model_dir = os.path.join(models_dir, model_size)
    base_url = "https://openaipublic.blob.core.windows.net/gpt-2/models"
    backup_base_url = "https://f001.backblazeb2.com/file/LLMs-from-scratch/gpt2"
    filenames = [
        "checkpoint", "encoder.json", "hparams.json",
        "model.ckpt.data-00000-of-00001", "model.ckpt.index",
        "model.ckpt.meta", "vocab.bpe"
    ]

    # Download files
    os.makedirs(model_dir, exist_ok=True)
    for filename in filenames:
        file_url = os.path.join(base_url, model_size, filename)
        backup_url = os.path.join(backup_base_url, model_size, filename)
        file_path = os.path.join(model_dir, filename)
        download_file(file_url, file_path, backup_url)

    # Load settings and params
    tf_ckpt_path = tf.train.latest_checkpoint(model_dir)
    settings = json.load(open(os.path.join(model_dir, "hparams.json")))
    params = load_gpt2_params_from_tf_ckpt(tf_ckpt_path, settings)

    return settings, params

In [4]:
settings, params = download_and_load_gpt2(model_size="124M", models_dir="gpt2")

checkpoint: 100%|██████████████████████████████████████████████████████████████████| 77.0/77.0 [00:00<00:00, 76.9kiB/s]
encoder.json: 100%|███████████████████████████████████████████████████████████████| 1.04M/1.04M [00:01<00:00, 842kiB/s]
hparams.json: 100%|████████████████████████████████████████████████████████████████| 90.0/90.0 [00:00<00:00, 43.5kiB/s]
model.ckpt.data-00000-of-00001: 100%|██████████████████████████████████████████████| 498M/498M [01:14<00:00, 6.67MiB/s]
model.ckpt.index: 100%|██████████████████████████████████████████████████████████| 5.21k/5.21k [00:00<00:00, 2.61MiB/s]
model.ckpt.meta: 100%|██████████████████████████████████████████████████████████████| 471k/471k [00:00<00:00, 536kiB/s]
vocab.bpe: 100%|████████████████████████████████████████████████████████████████████| 456k/456k [00:00<00:00, 623kiB/s]


In [65]:
# 1. Importação das bibliotecas
import torch
import torch.nn as nn

In [67]:
# 2. Configuração do dispositivo
device = "cuda" if torch.cuda.is_available() else "cpu"

In [69]:
# 3. Configuração do modelo GPT-2 otimizada
GPT_CONFIG = {
    "vocab_size": 50257,  # Tamanho do vocabulário
    "context_length": 256,  # Comprimento máximo do contexto
    "emb_dim": 768,  # Dimensão do embedding
    "n_heads": 12,  # Número de cabeças de atenção
    "n_layers": 12,  # Número de camadas
    "drop_rate": 0.1,  # Taxa de dropout
    "qkv_bias": True  # Uso de bias nas projeções de atenção
}

In [73]:
# 4. Inicialização do tokenizer
tokenizer = tiktoken.get_encoding("gpt2")

def encode_text(text):
    return torch.tensor(tokenizer.encode(text), dtype=torch.long).unsqueeze(0).to(device)

def decode_text(tokens):
    return tokenizer.decode(tokens.tolist())

In [75]:
# 5. Construção do modelo GPT
class GPTModel(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.tok_emb = nn.Embedding(config["vocab_size"], config["emb_dim"])
        self.pos_emb = nn.Embedding(config["context_length"], config["emb_dim"])
        self.drop_emb = nn.Dropout(config["drop_rate"])
        self.trf_blocks = nn.Sequential(*[TransformerBlock(config) for _ in range(config["n_layers"])] )
        self.final_norm = LayerNorm(config["emb_dim"])
        self.out_head = nn.Linear(config["emb_dim"], config["vocab_size"], bias=False)
    def forward(self, in_idx):
        batch_size, seq_len = in_idx.shape
        tok_embeds = self.tok_emb(in_idx)
        pos_embeds = self.pos_emb(torch.arange(seq_len, device=in_idx.device))
        x = tok_embeds + pos_embeds
        x = self.drop_emb(x)
        x = self.trf_blocks(x)
        x = self.final_norm(x)
        return self.out_head(x)

In [61]:
# 6. Construção do modelo GPT
class GPTModel(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.tok_emb = nn.Embedding(config["vocab_size"], config["emb_dim"])
        self.pos_emb = nn.Embedding(config["context_length"], config["emb_dim"])
        self.drop_emb = nn.Dropout(config["drop_rate"])
        self.trf_blocks = nn.Sequential(*[TransformerBlock(config) for _ in range(config["n_layers"])] )
        self.final_norm = LayerNorm(config["emb_dim"])
        self.out_head = nn.Linear(config["emb_dim"], config["vocab_size"], bias=False)
    def forward(self, in_idx):
        batch_size, seq_len = in_idx.shape
        tok_embeds = self.tok_emb(in_idx)
        pos_embeds = self.pos_emb(torch.arange(seq_len, device=in_idx.device))
        x = tok_embeds + pos_embeds
        x = self.drop_emb(x)
        x = self.trf_blocks(x)
        x = self.final_norm(x)
        return self.out_head(x)

In [77]:
# 6. Geração de texto aprimorada
def generate_text(model, tokenizer, prompt, max_new_tokens=20, temperature=1.0, top_k=50):
    model.to(device)
    model.eval()
    input_tokens = encode_text(prompt)
    generated_tokens = input_tokens.tolist()[0]
    
    for _ in range(max_new_tokens):
        with torch.no_grad():
            logits = model(input_tokens)[:, -1, :] / temperature  # Ajuste de temperatura
            logits = torch.topk(logits, top_k)[0]  # Aplica top-k sampling
            probs = torch.nn.functional.softmax(logits, dim=-1)
            next_token = torch.multinomial(probs, num_samples=1).item()
            generated_tokens.append(next_token)
            input_tokens = torch.tensor([generated_tokens], dtype=torch.long).to(device)
    
    output_text = decode_text(torch.tensor(generated_tokens))
    print("Output text:\n", output_text)

# Exemplo de Entrada: Hino Nacional Brasileiro
prompt = "Ouviram do Ipiranga as margens plácidas"
generate_text(GPTModel(GPT_CONFIG).to(device), tokenizer, prompt)


Output text:
 Ouviram do Ipiranga as margens plácidasO%01>*HI'MQD"+N(-IKM
